In [1]:
import sys
from tqdm import tqdm
import numpy as np
import pandas as pd
import keras.backend as k
import tensorflow as tf
sys.path.insert(0, "/home/mengdi/yuxiang.ye/Dn_development/")

/home/mengdi/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from Nodule_Detect.model.densenet_pytorch import DenseNet
from Nodule_Detect.model.densenet_keras import get_model

In [3]:
import torch
import torchvision.models as models
import hiddenlayer as hl

In [4]:
# weights = "/devdata/model_weights/pytorch_fp_model/pytorch_11_14/model_best.pth.tar"
# weights = "/devdata/model_weights/pytorch_fp_model/pytorch_11_16_dice_loss_4684_2222/model_best.pth.tar"
weights = "/devdata/model_weights/pytorch_fp_model/pytorch_20180410_tmp/model_best.pth.tar"

In [5]:
model = DenseNet(block_down_config = (2, 6, 8, 4), block_up_config=(4, 8, 6, 2))
model.load_state_dict(torch.load(weights, map_location='cpu')['state_dict'])
# model.cuda()

In [6]:
pytorch_state_dict = model.state_dict()

In [7]:
for name, weights in model.state_dict().items():
    print(name, weights.shape, weights.size())

firstconv.weight torch.Size([8, 1, 3, 3, 3]) torch.Size([8, 1, 3, 3, 3])
firstconv.bias torch.Size([8]) torch.Size([8])
denseBlocksDown.0.layers.0.norm1.weight torch.Size([8]) torch.Size([8])
denseBlocksDown.0.layers.0.norm1.bias torch.Size([8]) torch.Size([8])
denseBlocksDown.0.layers.0.norm1.running_mean torch.Size([8]) torch.Size([8])
denseBlocksDown.0.layers.0.norm1.running_var torch.Size([8]) torch.Size([8])
denseBlocksDown.0.layers.0.norm1.num_batches_tracked torch.Size([]) torch.Size([])
denseBlocksDown.0.layers.0.conv1.weight torch.Size([16, 8, 1, 1, 1]) torch.Size([16, 8, 1, 1, 1])
denseBlocksDown.0.layers.0.norm2.weight torch.Size([16]) torch.Size([16])
denseBlocksDown.0.layers.0.norm2.bias torch.Size([16]) torch.Size([16])
denseBlocksDown.0.layers.0.norm2.running_mean torch.Size([16]) torch.Size([16])
denseBlocksDown.0.layers.0.norm2.running_var torch.Size([16]) torch.Size([16])
denseBlocksDown.0.layers.0.norm2.num_batches_tracked torch.Size([]) torch.Size([])
denseBlocksDow

In [8]:
pytorch_layers = list(set([".".join(name.split(".")[:-1]) for name, *_ in model.state_dict().items()]))

In [9]:
pytorch_layers_flatten = list([name for name, *_ in model.state_dict().items()])

In [10]:
keras_model = get_model(weights=None, down_structure = [2, 6, 8, 4], up_structure = [4, 8, 6, 2])

up_structure
down_structure


In [11]:
len(pytorch_layers_flatten)

575

In [12]:
keras_layers = [getattr(layer, "name") for layer in keras_model.layers if len(getattr(layer, "weights"))]

In [13]:
print("keras_layer:", len(keras_layers), "pytorch_layers:", len(pytorch_layers))
if not len(set(keras_layers) ^ set(pytorch_layers)):
    print("the layers in keras same with pytorch!!!!!!!!!")
else:
    raise RuntimeError("keras and pytorch is not same, please check")

keras_layer: 192 pytorch_layers: 192
the layers in keras same with pytorch!!!!!!!!!


In [14]:
lay_num = 0
for layer in keras_model.layers:
    weights_list = []
    if layer.name in pytorch_layers:
        for index, i in enumerate(layer.weights):
            lay_num += 1
            print(i.name, "------>", i.shape)
            # convolution
            if ("conv" in i.name) or ("Conv" in i.name):
                if "kernel" in i.name:
                    layer_name = i.name.replace("/kernel:0", ".weight")
                elif "bias" in i.name:
                    layer_name = i.name.replace("/bias:0", ".bias")
                else:
                    raise RuntimeError("con layer unmatch！")
            # dense
            elif ("classifier" in i.name):
                if "kernel" in i.name:
                    layer_name = i.name.replace("/kernel:0", ".weight")
                elif "bias" in i.name:
                    layer_name = i.name.replace("/bias:0", ".bias")
                else:
                    raise RuntimeError("con layer unmatch！")
            # bn 
            elif ("norm" in i.name) or ("Bn" in i.name):
                if "gamma" in i.name:
                    layer_name = i.name.replace("/gamma:0", ".weight")
                elif "beta" in i.name:
                    layer_name = i.name.replace("/beta:0", ".bias")
                elif "moving_mean" in i.name:
                    layer_name = i.name.replace("/moving_mean:0", ".running_mean")
                elif "moving_variance" in i.name:
                    layer_name = i.name.replace("/moving_variance:0", ".running_var")
                else:
                    raise RuntimeError("norm layer unmatch！")
            else:
                raise RuntimeError(layer.name, "has no layer unmatch！")
                
            weights = pytorch_state_dict[layer_name].detach().numpy()
            if len(weights.shape) == 5:
                weights = np.transpose(weights, (2,3,4,1,0))
            elif len(weights.shape) == 2:
                weights = np.transpose(weights, (1,0))
            weights_list.append(np.round(weights, 4))
            print(layer_name)
            assert layer_name in pytorch_layers_flatten  
    layer.set_weights(weights_list)
print("lay_num:", lay_num)

firstconv/kernel:0 ------> (3, 3, 3, 1, 8)
firstconv.weight
firstconv/bias:0 ------> (8,)
firstconv.bias
denseBlocksDown.0.layers.0.norm1/gamma:0 ------> (8,)
denseBlocksDown.0.layers.0.norm1.weight
denseBlocksDown.0.layers.0.norm1/beta:0 ------> (8,)
denseBlocksDown.0.layers.0.norm1.bias
denseBlocksDown.0.layers.0.norm1/moving_mean:0 ------> (8,)
denseBlocksDown.0.layers.0.norm1.running_mean
denseBlocksDown.0.layers.0.norm1/moving_variance:0 ------> (8,)
denseBlocksDown.0.layers.0.norm1.running_var
denseBlocksDown.0.layers.0.conv1/kernel:0 ------> (1, 1, 1, 8, 16)
denseBlocksDown.0.layers.0.conv1.weight
denseBlocksDown.0.layers.0.norm2/gamma:0 ------> (16,)
denseBlocksDown.0.layers.0.norm2.weight
denseBlocksDown.0.layers.0.norm2/beta:0 ------> (16,)
denseBlocksDown.0.layers.0.norm2.bias
denseBlocksDown.0.layers.0.norm2/moving_mean:0 ------> (16,)
denseBlocksDown.0.layers.0.norm2.running_mean
denseBlocksDown.0.layers.0.norm2/moving_variance:0 ------> (16,)
denseBlocksDown.0.layers.0.no

denseBlocksDown.1.layers.5.norm2/gamma:0 ------> (16,)
denseBlocksDown.1.layers.5.norm2.weight
denseBlocksDown.1.layers.5.norm2/beta:0 ------> (16,)
denseBlocksDown.1.layers.5.norm2.bias
denseBlocksDown.1.layers.5.norm2/moving_mean:0 ------> (16,)
denseBlocksDown.1.layers.5.norm2.running_mean
denseBlocksDown.1.layers.5.norm2/moving_variance:0 ------> (16,)
denseBlocksDown.1.layers.5.norm2.running_var
denseBlocksDown.1.layers.5.conv2/kernel:0 ------> (3, 3, 3, 16, 4)
denseBlocksDown.1.layers.5.conv2.weight
transDownBlocks.1.norm/gamma:0 ------> (40,)
transDownBlocks.1.norm.weight
transDownBlocks.1.norm/beta:0 ------> (40,)
transDownBlocks.1.norm.bias
transDownBlocks.1.norm/moving_mean:0 ------> (40,)
transDownBlocks.1.norm.running_mean
transDownBlocks.1.norm/moving_variance:0 ------> (40,)
transDownBlocks.1.norm.running_var
transDownBlocks.1.conv/kernel:0 ------> (1, 1, 1, 40, 40)
transDownBlocks.1.conv.weight
transDownBlocks.1.conv/bias:0 ------> (40,)
transDownBlocks.1.conv.bias
dense

denseBlocksDown.2.layers.7.conv1/kernel:0 ------> (1, 1, 1, 68, 16)
denseBlocksDown.2.layers.7.conv1.weight
denseBlocksDown.2.layers.7.norm2/gamma:0 ------> (16,)
denseBlocksDown.2.layers.7.norm2.weight
denseBlocksDown.2.layers.7.norm2/beta:0 ------> (16,)
denseBlocksDown.2.layers.7.norm2.bias
denseBlocksDown.2.layers.7.norm2/moving_mean:0 ------> (16,)
denseBlocksDown.2.layers.7.norm2.running_mean
denseBlocksDown.2.layers.7.norm2/moving_variance:0 ------> (16,)
denseBlocksDown.2.layers.7.norm2.running_var
denseBlocksDown.2.layers.7.conv2/kernel:0 ------> (3, 3, 3, 16, 4)
denseBlocksDown.2.layers.7.conv2.weight
transDownBlocks.2.norm/gamma:0 ------> (72,)
transDownBlocks.2.norm.weight
transDownBlocks.2.norm/beta:0 ------> (72,)
transDownBlocks.2.norm.bias
transDownBlocks.2.norm/moving_mean:0 ------> (72,)
transDownBlocks.2.norm.running_mean
transDownBlocks.2.norm/moving_variance:0 ------> (72,)
transDownBlocks.2.norm.running_var
transDownBlocks.2.conv/kernel:0 ------> (1, 1, 1, 72, 72)

bottleneck.bottleneck.layers.2.conv1/kernel:0 ------> (1, 1, 1, 96, 16)
bottleneck.bottleneck.layers.2.conv1.weight
bottleneck.bottleneck.layers.2.norm2/gamma:0 ------> (16,)
bottleneck.bottleneck.layers.2.norm2.weight
bottleneck.bottleneck.layers.2.norm2/beta:0 ------> (16,)
bottleneck.bottleneck.layers.2.norm2.bias
bottleneck.bottleneck.layers.2.norm2/moving_mean:0 ------> (16,)
bottleneck.bottleneck.layers.2.norm2.running_mean
bottleneck.bottleneck.layers.2.norm2/moving_variance:0 ------> (16,)
bottleneck.bottleneck.layers.2.norm2.running_var
bottleneck.bottleneck.layers.2.conv2/kernel:0 ------> (3, 3, 3, 16, 4)
bottleneck.bottleneck.layers.2.conv2.weight
bottleneck.bottleneck.layers.3.norm1/gamma:0 ------> (100,)
bottleneck.bottleneck.layers.3.norm1.weight
bottleneck.bottleneck.layers.3.norm1/beta:0 ------> (100,)
bottleneck.bottleneck.layers.3.norm1.bias
bottleneck.bottleneck.layers.3.norm1/moving_mean:0 ------> (100,)
bottleneck.bottleneck.layers.3.norm1.running_mean
bottleneck.b

denseBlocksUp.1.layers.2.conv1/kernel:0 ------> (1, 1, 1, 96, 16)
denseBlocksUp.1.layers.2.conv1.weight
denseBlocksUp.1.layers.2.norm2/gamma:0 ------> (16,)
denseBlocksUp.1.layers.2.norm2.weight
denseBlocksUp.1.layers.2.norm2/beta:0 ------> (16,)
denseBlocksUp.1.layers.2.norm2.bias
denseBlocksUp.1.layers.2.norm2/moving_mean:0 ------> (16,)
denseBlocksUp.1.layers.2.norm2.running_mean
denseBlocksUp.1.layers.2.norm2/moving_variance:0 ------> (16,)
denseBlocksUp.1.layers.2.norm2.running_var
denseBlocksUp.1.layers.2.conv2/kernel:0 ------> (3, 3, 3, 16, 4)
denseBlocksUp.1.layers.2.conv2.weight
denseBlocksUp.1.layers.3.norm1/gamma:0 ------> (100,)
denseBlocksUp.1.layers.3.norm1.weight
denseBlocksUp.1.layers.3.norm1/beta:0 ------> (100,)
denseBlocksUp.1.layers.3.norm1.bias
denseBlocksUp.1.layers.3.norm1/moving_mean:0 ------> (100,)
denseBlocksUp.1.layers.3.norm1.running_mean
denseBlocksUp.1.layers.3.norm1/moving_variance:0 ------> (100,)
denseBlocksUp.1.layers.3.norm1.running_var
denseBlocksUp

denseBlocksUp.2.layers.2.conv2/kernel:0 ------> (3, 3, 3, 16, 4)
denseBlocksUp.2.layers.2.conv2.weight
denseBlocksUp.2.layers.3.norm1/gamma:0 ------> (84,)
denseBlocksUp.2.layers.3.norm1.weight
denseBlocksUp.2.layers.3.norm1/beta:0 ------> (84,)
denseBlocksUp.2.layers.3.norm1.bias
denseBlocksUp.2.layers.3.norm1/moving_mean:0 ------> (84,)
denseBlocksUp.2.layers.3.norm1.running_mean
denseBlocksUp.2.layers.3.norm1/moving_variance:0 ------> (84,)
denseBlocksUp.2.layers.3.norm1.running_var
denseBlocksUp.2.layers.3.conv1/kernel:0 ------> (1, 1, 1, 84, 16)
denseBlocksUp.2.layers.3.conv1.weight
denseBlocksUp.2.layers.3.norm2/gamma:0 ------> (16,)
denseBlocksUp.2.layers.3.norm2.weight
denseBlocksUp.2.layers.3.norm2/beta:0 ------> (16,)
denseBlocksUp.2.layers.3.norm2.bias
denseBlocksUp.2.layers.3.norm2/moving_mean:0 ------> (16,)
denseBlocksUp.2.layers.3.norm2.running_mean
denseBlocksUp.2.layers.3.norm2/moving_variance:0 ------> (16,)
denseBlocksUp.2.layers.3.norm2.running_var
denseBlocksUp.2.l

In [15]:
keras_model.save_weights("/devdata/model_weights/pytorch2keras/pytorch2keras_w_down2684_up4862.h5")